- We import pandas to read the excel and work with DataFrame
- We import requests for API requests
- pickle is to save variable
- time is to add a sleep between API calls if needed
- os.path is to check if a file exist

In [5]:
import pandas as pd
import requests
import pickle
import time
import os.path

We read the df previously saved in a file in order to get back the df in the workspace, or we load the excel file for the first execution

In [3]:
if os.path.isfile('savedDF.pickle') or os.path.isfile('Data-sheet-STUDENTS.xlsx'):
    if os.path.isfile('savedDF.pickle'):
        df = pd.read_pickle("savedDF.pickle")
        print("DF saved loaded")
    else:
        df = pd.read_excel('Data-sheet-STUDENTS.xlsx', header=2, index_col=0)
        print("DF from the empty execl loaded")
else:
    print("Error: no file to load the df exist in the current folder")

DF saved loaded


In [7]:
if os.path.isfile('indicatorsSession2.pickle'):
    with open('indicatorsSession2.pickle','rb') as fp:
        indicators = pickle.load(fp)
    print("Indicators loaded")
else:
    print("Indicators not loaded")
        

In [3]:
df["Country (according to CN Gov):"]

1                            Afghanistan
2                                Albania
3                                Armenia
4                             Azerbaijan
5                                Bahrain
6                             Bangladesh
7                                Belarus
8                                 Bhutan
9                 Bosnia and Herzegovina
10                                Brunei
11                              Bulgaria
12                              Cambodia
13                                 China
14                               Croatia
15                        Czech Republic
16                            East Timor
17                                 Egypt
18                               Estonia
19                      France (Not BRI)
20                               Georgia
21                     Germany (not BRI)
22                               Hungary
23                                 India
24                             Indonesia
25              

strip() clean begining and end of a string like "space" or "\n" ...

Next we get the isocode for each country

In [54]:
iso_codes = {}

In [55]:
for index, name in df["Country (according to CN Gov):"].items():    
    if name in iso_codes:
        continue
        
    clean_name = name.split(' (')[0]
    if name == "Thailiand":
        clean_name = "Thailand"
        
    if name == "The United Arab Emirates":
        clean_name = "United Arab Emirates"
        
    url = 'http://www.restcountries.eu/rest/v2/name/' + clean_name
    r = requests.get(url)
    if r.status_code == 200:
        data = r.json()
        country_code = data[0]['alpha3Code']
        iso_codes[name] = country_code
    else:
        print('[ERROR]['+ name +'] we cannot get the data, reason is ', r.status_code, r.text)

In [56]:
iso_codes

{'Afghanistan': 'AFG',
 'Albania': 'ALB',
 'Armenia': 'ARM',
 'Azerbaijan': 'AZE',
 'Bahrain': 'BHR',
 'Bangladesh': 'BGD',
 'Belarus': 'BLR',
 'Bhutan': 'BTN',
 'Bosnia and Herzegovina': 'BIH',
 'Brunei': 'BRN',
 'Bulgaria': 'BGR',
 'Cambodia': 'KHM',
 'China': 'CHN',
 'Croatia': 'HRV',
 'Czech Republic': 'CZE',
 'East Timor': 'TLS',
 'Egypt': 'EGY',
 'Estonia': 'EST',
 'France (Not BRI)': 'FRA',
 'Georgia': 'GEO',
 'Germany (not BRI)': 'DEU',
 'Hungary': 'HUN',
 'India': 'IOT',
 'Indonesia': 'IDN',
 'Iran': 'IRN',
 'Iraq': 'IRQ',
 'Israel': 'ISR',
 'Jordan': 'JOR',
 'Kazakhstan': 'KAZ',
 'Kuwait': 'KWT',
 'Kyrgyzstan': 'KGZ',
 'Laos': 'LAO',
 'Latvia': 'LVA',
 'Lebanon': 'LBN',
 'Lithuania': 'LTU',
 'Macedonia': 'MKD',
 'Malaysia': 'MYS',
 'Maldives': 'MDV',
 'Moldova': 'MDA',
 'Mongolia': 'MNG',
 'Montenegro': 'MNE',
 'Myanmar': 'MMR',
 'Nepal': 'NPL',
 'Oman': 'OMN',
 'Pakistan': 'PAK',
 'Philippines': 'PHL',
 'Poland': 'POL',
 'Qatar': 'QAT',
 'Romania': 'ROU',
 'Russia': 'RUS',
 

We put back the iso_code into the dataframe

In [58]:
df["isocode"] = df["Country (according to CN Gov):"].apply(lambda x: iso_codes[x])

Next we want to get the indicator for each country

In [4]:
dicoColumnIndicator = {'IMF WB\nCustoms and other import duties (% of tax revenue) 2016': 'GC.TAX.IMPT.ZS', 'WB IMF Non-performing loans to total gross loans ratio (%)': 'FB.AST.NPER.ZS', 'UIC Rail freight (mn ton x km travelled)': 'IS.RRS.GOOD.MT.K6', 'ICAO air freight (mn ton x km travelled)': 'IS.RRS.GOOD.MT.K6', 'WB Household final consumption expenditure, PPP bn USD': 'NE.CON.PRVT.PP.CD', 'WB UNESCO R&D Expenditure % of GDP': 'GB.XPD.RSDV.GD.ZS', 'WB UNESCO Researchers p. mn inhabitants': 'SP.POP.SCIE.RD.P6', 'IFDI Net Inflows (% of GDP)': 'BX.KLT.DINV.WD.GD.ZS', 'WHO Current Health Expenditure p.c., PPP USD': 'SH.XPD.CHEX.PP.CD', 'WB Research Expenditure % of GDP': 'GB.XPD.RSDV.GD.ZS', 'WIPO Trademark applications': 'IP.TMK.RSCT', 'WIPO Patent Applications': 'IP.PAT.RESD'}

In [6]:
indicators = {}

In [7]:
for nameColumn, valueIndicator in dicoColumnIndicator.items():
    
    if nameColumn not in indicators:
        indicators[nameColumn] = {}
                
    for index, isocode in df["isocode"].items():    
        if isocode in indicators[nameColumn]:
            continue

        url = 'http://api.worldbank.org/v2/country/' + isocode + '/indicator/'+ valueIndicator +'?format=json&mrv=1'
        r = requests.get(url)
        if r.status_code == 200:
            print(isocode)
            data = r.json()        
            try:
                indicators[nameColumn][isocode] = data[1][0]['value']
                print(indicators[nameColumn][isocode])
            except:
                print("[WARNING] " + nameColumn + " & " + isocode + " => No value found")
        else:
            print('[ERROR]['+ isocode +'] we cannot get the data, reason is ', r.status_code, r.text)
        
        #time.sleep(1) #delay 1s


AFG
NA
ALB
14
ARM
107
AZE
144
BHR
8
BGD
61
BLR
434
BTN
4
BIH
87
BRN
8
BGR
202
KHM
2
CHN
1245709
HRV
148
CZE
794
TLS
NA
EGY
918
EST
37
FRA
14415
GEO
75
DEU
47785
HUN
496
IOT
NA
IDN
2271
IRN
15264
IRQ
613
ISR
1436
JOR
26
KAZ
1055
KWT
NA
KGZ
137
LAO
NA
LVA
90
LBN
110
LTU
81
MKD
42
MYS
1166
MDV
NA
MDA
73
MNG
124
MNE
10
MMR
NA
NPL
20
OMN
4
PAK
193
PHL
323
POL
3924
QAT
19
ROU
1098
RUS
22777
SAU
909
SRB
171
SGP
1609
SVK
183
SVN
470
LKA
277
SYR
120
TJK
2
THA
979
ARE
52
TUR
8175
TKM
44
GBR
13301
USA
293904
UKR
2283
UZB
357
VNM
592
YEM
15


Insert the data into the dataFrame

In [10]:
for nameColumn, indicator in indicators.items():
    df[nameColumn] = df["isocode"].apply(lambda x: indicator[x])

Creating the new excel with the data

In [128]:
df.to_excel('Data-sheet-STUDENTS-filled.xlsx')

We save the df and indicators to read them next time we continue the work. 

In [12]:
df.to_pickle("savedDF.pickle")
with open('indicatorsSession2.pickle','wb') as fp:
    pickle.dump(a,fp)

In [64]:
df.columns

Index(['Country (according to CN Gov):', 'Date of research inquiry',
       'HF Index of Economic Freedom (Scale: 1-100)',
       'Heritage Foundation Index of Economic Freedom (Global Ranking)',
       'Bertelsmann Transformation index (BTI)',
       'Bertelsmann Transformation index (BTI) - Ranking',
       'IMF WB\nCustoms and other import duties (% of tax revenue) 2016',
       'WB IMF Non-performing loans to total gross loans ratio (%)',
       'Moody's Socvereign Ratings List', 'S&P Sovereign Ratings List',
       'Fitch Sovereign Ratings List',
       'UIC Rail freight (mn ton x km travelled)',
       'ICAO air freight (mn ton x km travelled)',
       'WB Household final consumption expenditure, PPP bn USD',
       'WB UNESCO R&D Expenditure % of GDP',
       'WB UNESCO Researchers p. mn inhabitants',
       'TI Corruption Perception Index',
       'TI Corruption Perception Index - Ranking', 'BRI GDP', 'BRI GDP p.c.',
       'BRI GDP growth rate', 'BRI Y.o.Y. GDP growth rate p.c

In [ ]:
df

In [ ]:
indicators